In [29]:
import numpy as np 
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch import optim
import cv2
import csv
import sys
import os
from PIL import Image
import numpy as np
import torch
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, ToTensor
from tqdm.notebook import tqdm
from torch.utils.data.sampler import SubsetRandomSampler
#from tqdm.autonotebook import tqdm
#from tqdm import tqdm


In [30]:
#加上   nn.Dropout2d(0.3), 去掉 norm2d
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()

        # define: encoder
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(3, 16, 3, 2),
            nn.Dropout2d(0.4),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(2),
            nn.LeakyReLU(0.01, inplace=True),
            nn.Conv2d(16, 32, 3, 2),
            nn.Dropout2d(0.3),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.01, inplace=True),
            nn.Conv2d(32, 64, 3, 1),
            nn.Dropout2d(0.2),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.01, inplace=True)
        )
 
        self.classifier = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(32, 10),
        )


    
    def forward(self, x):
        features = self.feature_extractor(x)
        features = features.view(x.shape[0], -1)
        logits = self.classifier(features)
        return logits

In [35]:
# detect is gpu available.
use_gpu = torch.cuda.is_available()
if use_gpu:
    device =torch.device('cuda:0')
else:
    device =torch.device("cpu")
class Image(Dataset):
    def __init__(self,label,data):
        self.data_path = data
        self.label_path = label
    def __getitem__(self,index):

        trainX = np.load(self.data_path)
        trainY = np.load(self.label_path)
        trainX = np.transpose(trainX, (0, 3, 1, 2))
        trainX = torch.Tensor(trainX)
        trainY = torch.Tensor(trainY)
        return trainX[index],trainY[index]

    def __len__(self):
        return len(np.load(self.label_path))

train_data = Image('./trainY.npy','./trainX.npy')



In [36]:
batch_size = 32
validation_split = .2
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(train_data)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_dataloader = DataLoader(train_data, batch_size=batch_size,sampler=train_sampler)
val_dataloader = DataLoader(train_data, batch_size=batch_size,sampler=valid_sampler)

In [37]:
def do_epoch(model, dataloader, criterion, optim=None):
    total_loss = 0
    total_accuracy = 0
    for x,y in (tqdm(dataloader, leave=False)):
        y_pred = model(x)
        y = y.long()
        loss = criterion(y_pred,y)
        if optim is not None:
            optim.zero_grad()
            loss.backward()
            optim.step()
        total_loss += loss.item()
        total_accuracy += (y_pred.max(1)[1] == y).float().mean().item()
    mean_loss = total_loss / len(dataloader)
    mean_accuracy = total_accuracy / len(dataloader)

    return mean_loss, mean_accuracy



#train_loader, val_loader = create_dataloaders(args.batch_size)
train_dataloader  
model = Net()
optim = torch.optim.Adam(model.parameters())
lr_schedule = torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=1, verbose=True)
criterion = torch.nn.CrossEntropyLoss()
best_accuracy = 0
for epoch in range(500):
    model.train()
    train_loss, train_accuracy = do_epoch(model, train_dataloader, criterion, optim=optim)
    model.eval()
    with torch.no_grad():
        val_loss, val_accuracy = do_epoch(model, val_dataloader, criterion, optim=None)

    tqdm.write(f'EPOCH {epoch:03d}: train_loss={train_loss:.4f}, train_accuracy={train_accuracy:.4f} '
               f'val_loss={val_loss:.4f}, val_accuracy={val_accuracy:.4f}')
    if val_accuracy > best_accuracy:
        print('Saving model...')
        best_accuracy = val_accuracy
        torch.save(model.state_dict(), 'trained_models/source.pt')

    lr_schedule.step(val_loss)

EPOCH 000: train_loss=2.1100, train_accuracy=0.2432 val_loss=1.8921, val_accuracy=0.3525


EPOCH 001: train_loss=1.7432, train_accuracy=0.3882 val_loss=1.6120, val_accuracy=0.4443


EPOCH 002: train_loss=1.5449, train_accuracy=0.4735 val_loss=1.6610, val_accuracy=0.4258


EPOCH 003: train_loss=1.4254, train_accuracy=0.5138 val_loss=1.5467, val_accuracy=0.4639


EPOCH 004: train_loss=1.3230, train_accuracy=0.5465 val_loss=1.4365, val_accuracy=0.5039


...

EPOCH 006: train_loss=1.1692, train_accuracy=0.6028 val_loss=1.4740, val_accuracy=0.4814


EPOCH 007: train_loss=1.1309, train_accuracy=0.6222 val_loss=1.4717, val_accuracy=0.5088


EPOCH 008: train_loss=1.0597, train_accuracy=0.6472 val_loss=1.2772, val_accuracy=0.5625


EPOCH 009: train_loss=1.0345, train_accuracy=0.6462 val_loss=1.3862, val_accuracy=0.5283


KeyboardInterrupt: 

In [5]:
torch.save(model.state_dict(), './model.pt')

In [22]:
test_data = np.load('testX.npy')


rgb_test_data = []
for image in test_data:
    padding =np.zeros((image.shape[0]+2,image.shape[1]+2),dtype = np.uint8)
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            padding[i+1][j+1] = image[i][j]
    color_img = cv2.cvtColor(padding,cv2.COLOR_GRAY2RGB)
    rgb_test_data.append(color_img)
rgb_test_data = np.array(rgb_test_data)
test_data = np.transpose(rgb_test_data, (0, 3, 1, 2))

print(test_data.shape)

(100000, 3, 30, 30)


In [26]:
test_data = torch.Tensor(test_data)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)
model.eval()
ans = []
for idx ,img in enumerate(test_loader):
    ans.append(model(img))
ans_final =[]
for a in ans:
    for pre in a:
        pre = pre.tolist()
        ans_final.append(pre.index(max(pre)))
    

In [27]:
with open("only_Pre_train.csv","w") as f:
    print("id,label", file = f)
    for id,label in enumerate(ans_final):
        print("{},{}".format(id,label) ,file = f)